In [1]:
from pathlib import Path
from typing import List, Tuple
from tempfile import TemporaryDirectory

import pandas as pd
from spacy import displacy

from meddocan.data import meddocan_zip, ArchiveFolder
from meddocan.data.containers import BratAnnotations, BratFilesPair, BratSpan
from meddocan.data.docs_iterators import GsDocs
from meddocan.language.pipeline import meddocan_pipeline
from meddocan.data import meddocan_url, meddocan_zip, ArchiveFolder
from meddocan.data.docs_iterators import BratAnnotations, GsDocs
from meddocan.data.utils import set_ents_from_brat_spans
from meddocan.data.corpus import MEDDOCAN

# MEDDOCAN: Anonimización aplicada al ámbito médico

## Introducción

* Medical Document Anonymization Track (track 9 of the <abbr title="Iberian Languages Evaluation Forum 2019"> [IberLEF 2019](http://ceur-ws.org/Vol-2421/)).</abbr>.

    Detectar automaticamente la información sanitaria protegida (PHI)

* ¿Como?

    Name Entity Recognition con methodos de Transfer Learning basados en los transformadores


## Dataset

* Visualización de los datos

![Figure 1: An example of MEDDOCAN annotation visualized using the BRAT annotation interface.](https://temu.bsc.es/meddocan/wp-content/uploads/2019/03/image-1-1024x922.png)

* Distribución del tipo de entidad entre los juegos de datos

|               Tipo               | Train | Dev | Test | Total |
| :------------------------------: | :---: | :-: | :--: | :---: |
|            TERRITORIO            | 1875  | 987 | 956  | 3818  |
|              FECHAS              | 1231  | 724 | 611  | 2566  |
|      EDAD SUJETO ASISTENCIA      | 1035  | 521 | 518  | 2074  |
|     NOMBRE SUJETO ASISTENCIA     | 1009  | 503 | 502  | 2014  |
|    NOMBRE PERSONAL SANITARIO     | 1000  | 497 | 501  | 1998  |
|      SEXO SUJETO ASISTENCIA      |  925  | 455 | 461  | 1841  |
|              CALLE               |  862  | 434 | 413  | 1709  |
|               PAIS               |  713  | 347 | 363  | 1423  |
|       ID SUJETO ASISTENCIA       |  567  | 292 | 283  | 1142  |
|        CORREO ELECTRONICO        |  469  | 241 | 249  |  959  |
| ID TITULACION PERSONAL SANITARIO |  471  | 226 | 234  |  931  |
|         ID ASEGURAMIENTO         |  391  | 194 | 198  |  783  |
|             HOSPITAL             |  255  | 140 | 130  |  525  |
|   FAMILIARES SUJETO ASISTENCIA   |  243  | 92  |  81  |  416  |
|           INSTITUCION            |  98   | 72  |  67  |  237  |
|     ID CONTACTO ASISTENCIAL      |  77   | 32  |  39  |  148  |
|         NUMERO TELEFONO          |  58   | 25  |  26  |  109  |
|            PROFESION             |  24   |  4  |  9   |  37   |
|            NUMERO FAX            |  15   |  6  |  7   |  28   |
|     OTROS SUJETO ASISTENCIA      |   9   |  6  |  7   |  22   |
|           CENTRO SALUD           |   6   |  2  |  6   |  14   |
|   ID EMPLEO PERSONAL SANITARIO   |   0   |  1  |  0   |   1   |
| IDENTIF VEHICULOS NRSERIE PLACAS |   0   |  0  |  0   |   0   |
|   IDENTIF DISPOSITIVOS NRSERIE   |   0   |  0  |  0   |   0   |
|     NUMERO BENEF PLAN SALUD      |   0   |  0  |  0   |   0   |
|             URL WEB              |   0   |  0  |  0   |   0   |
|       DIREC PROT INTERNET        |   0   |  0  |  0   |   0   |
|        IDENTF BIOMETRICOS        |   0   |  0  |  0   |   0   |
|       OTRO NUMERO IDENTIF        |   0   |  0  |  0   |   0   |

* **Código**

El paquete ``meddocan.data`` proporciona clases y funciones para tratar con los conjuntos de datos originales de
meddocan en la web.

- `meddocan.data.meddocan_url` contiene el enlace url para llegar a las carpetas de datos comprimidas.

In [2]:
meddocan_url

MeddocanUrl(sample='http://temu.bsc.es/meddocan/wp-content/uploads/2019/03/sample-set.zip', train='http://temu.bsc.es/meddocan/wp-content/uploads/2019/03/train-set.zip', dev='http://temu.bsc.es/meddocan/wp-content/uploads/2019/04/dev-set-1.zip', test='http://temu.bsc.es/meddocan/wp-content/uploads/2019/05/test-set.zip', background='http://temu.bsc.es/meddocan/wp-content/uploads/2019/10/background-set.zip', base='http://temu.bsc.es/meddocan/wp-content/uploads/2019')

* `meddocan.data.meddocan_zip` obtiene el par de archivos brat para
una carpeta determinada del conjunto de datos en caché a través de su método ``brat_files``.

In [3]:
brat_files_pairs = meddocan_zip.brat_files(ArchiveFolder.train)

In [4]:
brat_files_pair = next(brat_files_pairs)

In [5]:
brat_files_pair.txt

Path('/home/wave/.meddocan/datasets/meddocan/train-set.zip', 'train/brat/S0004-06142005000500011-1.txt')

In [6]:
brat_files_pair.ann

Path('/home/wave/.meddocan/datasets/meddocan/train-set.zip', 'train/brat/S0004-06142005000500011-1.ann')

Visualizamos los datos brutos

In [7]:
brat_annotations = BratAnnotations.from_brat_files(brat_files_pair)

- El texto

In [11]:
text = brat_annotations.text; text[:200]

'Datos del paciente.\nNombre:  Ernesto.\nApellidos: Rivera Bueno.\nNHC: 368503.\nNASS: 26 63514095.\nDomicilio:  Calle Miguel Benitez 90.\nLocalidad/ Provincia: Madrid.\nCP: 28016.\nDatos asistenciales.\nFecha '

Las anotaciones

In [13]:
brat_spans = brat_annotations.brat_spans; brat_spans[:5]

[BratSpan(id='T1', entity_type='FECHAS', start=215, end=225, text='03/03/1946'),
 BratSpan(id='T2', entity_type='CORREO_ELECTRONICO', start=2421, end=2439, text='nnavcu@hotmail.com'),
 BratSpan(id='T3', entity_type='PAIS', start=2406, end=2412, text='España'),
 BratSpan(id='T4', entity_type='TERRITORIO', start=2398, end=2404, text='Madrid'),
 BratSpan(id='T5', entity_type='TERRITORIO', start=2392, end=2397, text='28036')]

### Preprocessamiento

Para empezar llamamos nuestro pipeline ``meddocan.language.pipeline.meddocan_pipeline`` hecho con [spaCY](https://spacy.io/) y miramos sus componentes.

In [14]:
nlp = meddocan_pipeline()
pd.DataFrame(nlp.pipe_names, columns=["componentes"]).T

,0,1,2,3
componentes,missaligned_splitter,line_sentencizer,predictor,write_methods


Vamos a hacer pasar nuestro texto de ejemplo por el ``meddocan_pipeline`` y ver el efecto de cada componente.

In [15]:
doc = nlp(text)

#### Tokenización

In [16]:
pd.DataFrame([token.text for token in doc[:10]], columns=["token"]).T

,0,1,2,3,4,5,6,7,8,9
token,Datos,del,paciente,.,\n,Nombre,:,,Ernesto,.


#### Casos especiales

In [17]:
[token for token in nlp("DREnric Lopez")]

[DR, Enric, Lopez]

#### Partición del documento en párafos

In [18]:
pd.DataFrame([sent.text for sent in doc.sents][:4], columns=["párafo"])


,párafo
0,Datos del paciente.\n
1,Nombre: Ernesto.\n
2,Apellidos: Rivera Bueno.\n
3,NHC: 368503.\n


#### predictor y entidades

En una primera fase no tenemos modelos con el cual hacer predicciones.
Entonces añadimos la entidades a nuestro objeto ``doc`` usando la function ``meddocan.data.utils.set_ents_from_brat_spans``.

In [19]:
doc = set_ents_from_brat_spans(doc, brat_spans=brat_spans)
displacy.render(doc[:100], style="ent")

#### Write methods

Para la evaluación, se puede escribir los datos al formato **Brat**

In [20]:
with TemporaryDirectory() as td:
    pth = Path(td, "file.txt")
    doc._.to_ann(pth)
    for i, line in enumerate(pth.read_text().split("\n")):
        print(line)
        if i > 10:
            break

T_0	NOMBRE_SUJETO_ASISTENCIA 29 36	Ernesto
T_1	NOMBRE_SUJETO_ASISTENCIA 49 61	Rivera Bueno
T_2	ID_SUJETO_ASISTENCIA 68 74	368503
T_3	ID_ASEGURAMIENTO 82 93	26 63514095
T_4	CALLE 107 130	Calle Miguel Benitez 90
T_5	TERRITORIO 154 160	Madrid
T_6	TERRITORIO 166 171	28016
T_7	FECHAS 215 225	03/03/1946
T_8	PAIS 233 239	España
T_9	EDAD_SUJETO_ASISTENCIA 247 254	70 años
T_10	SEXO_SUJETO_ASISTENCIA 261 262	H
T_11	FECHAS 282 292	12/12/2016


Para el entrenamiento, los datos se escriben al formato **conll03**.

In [21]:
with TemporaryDirectory() as td:
    pth = Path(td, "file.txt")
    doc._.to_connl03(pth)
    for i, line in enumerate(pth.read_text().split("\n")):
        print(line)
        if i > 10:
            break

Datos O
del O
paciente O
. O

Nombre O
: O
Ernesto B-NOMBRE_SUJETO_ASISTENCIA
. O

Apellidos O
: O


## Entrenamiento con Flair

### Creación del dataset ``MEDDOCAN``

Los distinctos conjunto de datos

In [22]:
pd.DataFrame([folder for folder in ArchiveFolder], columns=["sets"])

,sets
0,ArchiveFolder.train
1,ArchiveFolder.dev
2,ArchiveFolder.test
3,ArchiveFolder.sample
4,ArchiveFolder.background


Solo se usan los sets de *train*, *dev* y *test*.

Utilizamos el objecto ``meddocan.data.docs_iterators.GsDocs`` que nos permitte obtener objectos ``Doc`` usando el ``meddocan_pipeline`` para cada una de los conjuntos de datos.

In [23]:
gs_docs = GsDocs(ArchiveFolder.train)
docs_with_brat_pair = iter(gs_docs)
doc_with_brat_pair = next(docs_with_brat_pair)

``doc_with_brat_pair`` ayuda a crear el ``MEDDOCAN`` corpus que hereda de ``flair.datasets.ColumnCorpus``.

El ``MEDDOCAN`` corpus esta creado a partir de ficheros temporales que contienen el texto asi como los offsets al formato **connl03**.

In [24]:
corpus = MEDDOCAN(sentences=True, in_memory=True, document_separator_token="-DOCSTART-")

2023-02-27 10:54:13,435 Reading data from /tmp/tmpc7969u4s
2023-02-27 10:54:13,435 Train: /tmp/tmpc7969u4s/train
2023-02-27 10:54:13,436 Dev: /tmp/tmpc7969u4s/dev
2023-02-27 10:54:13,436 Test: /tmp/tmpc7969u4s/test


In [26]:
print(corpus)

Corpus: 10811 train + 5518 dev + 5405 test sentences


Ahora podemos pasar al entrenamiento!

### Transfert learning

Comparación entre el aprendizaje supervisado tradicional (izquierda) y el transfer learning (derecha)

![transfer-learning](../cdti/figures/transformers-1.png)

### Flert ([Flair](https://github.com/flairNLP/flair))

![Flert](../cdti/figures/flert-1.png)

* **baseline**

    * Flair + LSTM +CRF

* **2 enfoques para el NER basados en Transformers**

    * Finetuning + linear transformation
    * Caractéristica + LSTM + CRF



* Entrenamiento: **Flair + LSTM-CRF**

![training flair](../cdti/figures/flair-1.png)

    * procedimiento de entrenamiento


|     Parameter      |       Value       |
| :----------------: | :---------------: |
|   Learning rate    |        0.1        |
|  Mini Batch size   |         4         |
|     Max epochs     |        150        |
|     Optimizer      |        SGD        |
|     Scheduler      | Anneal On Plateau |

* Entrenamiento: **Método de ajuste fino**

 Agrupación de subpalabras para crear representaciones a nivel de tokens que luego se pasan a la capa lineal final
![training finetuning](../cdti/figures/flert-2.png)

    * BETO

|     Parameter      |              Value              |
| :----------------: | :-----------------------------: |
| Transformer layers |              last               |
|   Learning rate    |              5e-6               |
|  Mini Batch size   |                4                |
|     Max epochs     |               150               |
|     Optimizer      |              AdamW              |
|     Scheduler      | Linear Warmup With Linear Decay |
|       Warmup       |               0.1               |
|  Subword pooling   |              first              |

    * XLM RoBERTa Large

|     Parameter      |              Value              |
| :----------------: | :-----------------------------: |
| Transformer layers |              last               |
|   Learning rate    |              5e-6               |
|  Mini Batch size   |                4                |
|     Max epochs     |               40                |
|     Optimizer      |              AdamW              |
|     Scheduler      | Linear Warmup With Linear Decay |
|       Warmup       |               0.1               |
|  Subword pooling   |              first              |

* Entrenamiento: **Método basado en características**

![training feature based](../cdti/figures/flert-3.png)

    * Procedimiento de entrenamiento

|     Parameter      |       Value       |
| :----------------: | :---------------: |
|   Learning rate    |        0.1        |
|  Mini Batch size   |         4         |
|     Max epochs     |        150        |
|     Optimizer      |        SGD        |
|     Scheduler      | Anneal On Plateau |

In [ ]:
!tensorboard --logdir ../experiments

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

I0227 11:13:45.097182 140381710694144 plugin.py:429] Monitor runs begin
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:6006/ (Press CTRL+C to quit)


* Entrenamiento: Método basado en características

### Entrenamiento

In [ ]:
!cd .. && bash scripts/test-cov.sh

## Evaluación

In [ ]:
gs_docs = GsDocs(ArchiveFolder.train)
docs_with_brat_pair = iter(gs_docs)
doc_with_brat_pair = next(docs_with_brat_pair)